# Growthcurves Tutorial

This tutorial is organized into two sections:

1. **Mechanistic** model fitting (ODE-based, parametric)
2. **Phenomenological** model fitting
   - **Parametric** phenomenological models (closed-form ln-space models)
   - **Non-parametric** phenomenological models (spline and sliding-window)

We'll fit each group, extract growth statistics, and visualize fitted curves.

In [1]:
import growthcurves as gc
import pandas as pd
import numpy as np

# Example bacterial growth data: OD600 measurements
# This represents a typical growth curve with lag, exponential, and stationary phases
data = [
    0.0493,
    0.0494,
    0.0492,
    0.0491,
    0.0491,
    0.0492,
    0.0492,
    0.0490,
    0.0491,
    0.0491,
    0.0492,
    0.0490,
    0.0492,
    0.0490,
    0.0490,
    0.0489,
    0.0491,
    0.0491,
    0.0491,
    0.0491,
    0.0491,
    0.0491,
    0.0492,
    0.0491,
    0.0492,
    0.0492,
    0.0492,
    0.0493,
    0.0492,
    0.0493,
    0.0495,
    0.0494,
    0.0494,
    0.0494,
    0.0494,
    0.0494,
    0.0495,
    0.0494,
    0.0495,
    0.0497,
    0.0496,
    0.0496,
    0.0498,
    0.0497,
    0.0498,
    0.0498,
    0.0498,
    0.0499,
    0.0500,
    0.0499,
    0.0501,
    0.0499,
    0.0501,
    0.0500,
    0.0499,
    0.0502,
    0.0502,
    0.0503,
    0.0502,
    0.0503,
    0.0503,
    0.0503,
    0.0504,
    0.0503,
    0.0504,
    0.0506,
    0.0506,
    0.0506,
    0.0506,
    0.0507,
    0.0508,
    0.0509,
    0.0509,
    0.0509,
    0.0510,
    0.0510,
    0.0510,
    0.0512,
    0.0512,
    0.0513,
    0.0514,
    0.0513,
    0.0514,
    0.0515,
    0.0515,
    0.0516,
    0.0515,
    0.0518,
    0.0518,
    0.0518,
    0.0520,
    0.0519,
    0.0520,
    0.0522,
    0.0521,
    0.0522,
    0.0523,
    0.0523,
    0.0525,
    0.0526,
    0.0527,
    0.0528,
    0.0528,
    0.0527,
    0.0530,
    0.0531,
    0.0531,
    0.0532,
    0.0534,
    0.0533,
    0.0535,
    0.0534,
    0.0537,
    0.0537,
    0.0540,
    0.0539,
    0.0540,
    0.0540,
    0.0543,
    0.0544,
    0.0545,
    0.0546,
    0.0547,
    0.0549,
    0.0549,
    0.0551,
    0.0552,
    0.0553,
    0.0554,
    0.0555,
    0.0557,
    0.0557,
    0.0559,
    0.0560,
    0.0560,
    0.0564,
    0.0565,
    0.0567,
    0.0567,
    0.0570,
    0.0571,
    0.0574,
    0.0575,
    0.0576,
    0.0579,
    0.0581,
    0.0582,
    0.0583,
    0.0585,
    0.0588,
    0.0591,
    0.0592,
    0.0594,
    0.0598,
    0.0600,
    0.0602,
    0.0606,
    0.0610,
    0.0613,
    0.0616,
    0.0620,
    0.0624,
    0.0627,
    0.0633,
    0.0638,
    0.0643,
    0.0649,
    0.0655,
    0.0659,
    0.0666,
    0.0672,
    0.0679,
    0.0685,
    0.0695,
    0.0703,
    0.0712,
    0.0722,
    0.0732,
    0.0741,
    0.0754,
    0.0766,
    0.0780,
    0.0792,
    0.0806,
    0.0821,
    0.0834,
    0.0853,
    0.0867,
    0.0885,
    0.0900,
    0.0920,
    0.0941,
    0.0962,
    0.0982,
    0.1002,
    0.1024,
    0.1044,
    0.1063,
    0.1078,
    0.1100,
    0.1120,
    0.1141,
    0.1163,
    0.1188,
    0.1214,
    0.1238,
    0.1262,
    0.1290,
    0.1319,
    0.1346,
    0.1376,
    0.1407,
    0.1438,
    0.1468,
    0.1502,
    0.1538,
    0.1576,
    0.1615,
    0.1653,
    0.1694,
    0.1735,
    0.1783,
    0.1834,
    0.1895,
    0.1949,
    0.2008,
    0.2072,
    0.2128,
    0.2194,
    0.2255,
    0.2321,
    0.2386,
    0.2451,
    0.2523,
    0.2590,
    0.2647,
    0.2700,
    0.2752,
    0.2812,
    0.2866,
    0.2920,
    0.2969,
    0.3022,
    0.3081,
    0.3130,
    0.3187,
    0.3233,
    0.3280,
    0.3328,
    0.3363,
    0.3409,
    0.3451,
    0.3486,
    0.3532,
    0.3570,
    0.3599,
    0.3634,
    0.3670,
    0.3703,
    0.3736,
    0.3773,
    0.3806,
    0.3845,
    0.3882,
    0.3933,
    0.3992,
    0.4032,
    0.4068,
    0.4108,
    0.4146,
    0.4177,
    0.4201,
    0.4236,
    0.4263,
    0.4288,
    0.4322,
    0.4347,
    0.4373,
    0.4394,
    0.4412,
    0.4426,
    0.4440,
    0.4461,
    0.4481,
    0.4487,
    0.4500,
    0.4514,
    0.4522,
    0.4534,
    0.4532,
    0.4541,
    0.4545,
    0.4552,
    0.4553,
    0.4557,
    0.4566,
    0.4561,
    0.4571,
    0.4579,
    0.4579,
    0.4588,
    0.4587,
    0.4600,
    0.4595,
    0.4601,
    0.4596,
    0.4597,
    0.4601,
    0.4603,
    0.4598,
    0.4596,
    0.4597,
    0.4595,
    0.4600,
    0.4607,
    0.4606,
    0.4606,
    0.4601,
    0.4605,
    0.4602,
    0.4650,
    0.4646,
    0.4637,
    0.4622,
    0.4609,
    0.4605,
    0.4601,
    0.4597,
    0.4605,
    0.4600,
    0.4598,
    0.4600,
    0.4604,
    0.4602,
    0.4605,
    0.4606,
    0.4611,
    0.4604,
    0.4605,
    0.4609,
    0.4604,
    0.4615,
    0.4613,
    0.4615,
    0.4615,
    0.4616,
    0.4616,
    0.4617,
    0.4622,
    0.4624,
    0.4621,
    0.4621,
    0.4625,
    0.4626,
    0.4625,
    0.4626,
    0.4627,
    0.4623,
    0.4628,
    0.4628,
    0.4625,
    0.4633,
    0.4632,
    0.4632,
    0.4641,
    0.4644,
    0.4645,
    0.4645,
    0.4646,
    0.4652,
    0.4661,
    0.4659,
    0.4662,
    0.4666,
    0.4661,
    0.4673,
    0.4675,
    0.4678,
    0.4674,
    0.4683,
    0.4683,
    0.4684,
    0.4693,
    0.4697,
    0.4697,
    0.4702,
    0.4699,
    0.4701,
    0.4715,
    0.4716,
    0.4718,
    0.4723,
    0.4724,
    0.4724,
    0.4734,
    0.4730,
    0.4734,
    0.4744,
    0.4738,
    0.4745,
    0.4750,
    0.4758,
    0.4762,
    0.4764,
    0.4766,
    0.4771,
    0.4775,
    0.4777,
    0.4784,
    0.4783,
    0.4787,
    0.4796,
    0.4800,
    0.4805,
    0.4812,
    0.4817,
    0.4825,
    0.4826,
    0.4826,
    0.4833,
    0.4837,
    0.4843,
    0.4842,
    0.4844,
    0.4851,
    0.4860,
    0.4867,
    0.4864,
    0.4872,
    0.4876,
    0.4880,
    0.4889,
    0.4896,
    0.4893,
    0.4901,
    0.4906,
    0.4907,
    0.4913,
]

# Create time array: measurements taken every 12 minutes, converted to hours
# Total duration: ~138.6 hours
time = np.array([(12 * n) / 60 for n in range(len(data))])

In [ ]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import pprint

## How Growth Parameters Are Calculated

The table below summarizes how the main reported growth statistics are calculated across model classes.

| Output key | Meaning | How it is calculated |
|---|---|---|
| `max_od` | Maximum observed/fitted OD | Maximum OD over the valid data range |
| `specific_growth_rate` | Maximum specific growth rate (μ_max) | Maximum of `d(ln N)/dt` from the fitted model (or local fit for non-parametric) |
| `intrinsic_growth_rate` | Intrinsic model rate parameter | For mechanistic models: fitted intrinsic `μ`; for phenomenological/non-parametric: `None` |
| `doubling_time` | Doubling time in hours | `ln(2) / specific_growth_rate` |
| `time_at_umax` | Time at maximum specific growth | Time where `specific_growth_rate` reaches its maximum |
| `od_at_umax` | OD at time of μ_max | Model-predicted OD at `time_at_umax` |
| `exp_phase_start`, `exp_phase_end` | Exponential phase boundaries | From threshold or tangent phase-boundary method in `extract_stats()` |
| `model_rmse` | Fit error | RMSE between observed OD and model-predicted OD over the model fit window |

For this tutorial:
- Mechanistic comparisons use mechanistic parametric fits.
- Phenomenological comparisons include both phenomenological parametric and non-parametric fits.


# Extract growth stats from the dataset

The `extract_stats_from_fit()` function calculates these key metrics:

- `max_od`: Maximum OD value within the fitted window
- `specific_growth_rate`: **Observed** maximum specific growth rate μ_max (hour⁻¹) - calculated from the fitted curve
- `intrinsic_growth_rate`: **Model parameter** for intrinsic growth rate (parametric models only, `None` for non-parametric)
- `doubling_time`: Time to double the population at peak growth (hours)
- `exp_phase_start`: When exponential phase begins (hours)
- `exp_phase_end`: When exponential phase ends (hours)
- `time_at_umax`: Time when μ reaches its maximum (hours)
- `od_at_umax`: OD value at time of maximum μ
- `fit_t_min`: Start of fitting window (hours)
- `fit_t_max`: End of fitting window (hours)
- `fit_method`: Identifier for the method used
- `model_rmse`: Root mean squared error

Descriptive parameters are extracted from the fits. Where parameters are not extracted directly from the fitted model, they are calculated. The table below shows how different stats are calculated according to the different approaches:

## MECHANISTIC MODELS

| Name | Model | Equation | Exp Start | Exp End | Intrinsic μ | μ max | Carrying Capacity | Fit |
|------|-------|----------|-----------|---------|-------------|-------|-------------------|-----|
| Logistic | parametric | `dN/dt = μ * (1 - N / K) * N` | threshold/tangent | threshold/tangent | μ | max dln(N)/dt | K | entire curve |
| Gompertz | parametric | `dN/dt = μ * math.log(K / N) * N` | threshold/tangent | threshold/tangent | μ | max dln(N)/dt | K | entire curve |
| Richards | parametric | `dN/dt = μ * (1 - (N / K)**beta) * N` | threshold/tangent | threshold/tangent | μ | max dln(N)/dt | A | entire curve |
| Baranyi | parametric | `dN/dt= μ * math.exp(μ * t) / (math.exp(h0) - 1 + math.exp(μ * t)) * (1 - N / K) * N` | threshold/tangent | threshold/tangent | μ | max dln(N)/dt | K | entire curve |

## PHENOMENOLOGICAL MODELS

| Name | Model | Equation | Exp Start | Exp End | Intrinsic μ | μ max | Max OD | Fit |
|------|-------|----------|-----------|---------|-------------|-------|--------|-----|
| Linear | non-parametric | `ln(N(t)) = N0 + b * t` | threshold/tangent | threshold/tangent | n.a. | b | max OD raw | only window |
| Spline | non-parametric | `ln(N(t)) = spline(t)` | threshold/tangent | threshold/tangent | n.a. | max of derivative of spline | max OD raw | only log phase |
| Logistic (phenom) | parametric | `ln(N(t)/N0) = A / (1 + exp(4 * μ_max * (λ - t) / A + 2))` | λ | threshold/tangent | n.a. | μ_max | K | entire curve |
| Gompertz (phenom) | parametric | `ln(N(t)/N0) = A * exp(-exp(μ_max * exp(1) * (λ - t) / A + 1))` | λ | threshold/tangent | n.a. | μ_max | K | entire curve |
| Gompertz (modified) | parametric | `ln(N(t)/N0) = A * exp(-exp(μ_max * exp(1) * (λ - t) / A + 1)) + A * exp(α * (t - t_shift))` | λ | threshold/tangent | n.a. | μ_max | K | entire curve |
| Richards (phenom) | parametric | `ln(N(t)/N0) = A * (1 + ν * exp(1 + ν + μ_max * (1 + ν)**(1/ν) * (λ - t) / A))**(-1/ν)` | λ | threshold/tangent | n.a. | μ_max | K | entire curve |

### Understanding Growth Rates: Intrinsic vs. Observed

**Important distinction:**

- **`specific_growth_rate`** (μ_max): The **observed** maximum specific growth rate calculated from the fitted curve as max(d(ln N)/dt). This is what you measure from the data.

- **`intrinsic_growth_rate`**: The **model parameter** representing intrinsic growth capacity:
  - **Parametric models**: This is a fitted parameter (e.g., `r` in Logistic, `mu_max` in Gompertz)
  - **Non-parametric methods**: Returns `None` (no model parameter exists)

Two methods are available for determining exponential phase boundaries:

#### 1. **Threshold Method** (used by parametric models)
- Tracks the instantaneous specific growth rate μ(t)
- `exp_phase_start`: First time when μ exceeds a fraction of μ_max (default: 15%)
- `exp_phase_end`: First time after peak when μ drops below the threshold (default: 15%)

#### 2. **Tangent Method** (used by non-parametric methods)
- Constructs a tangent line **in log space** at the point of maximum growth rate
- Extends this tangent to intersect baseline (exp_phase_start) and plateau (exp_phase_end)
- Tangent line equation: `ln(OD(t)) = ln(OD_umax) + μ_max * (t - t_umax)`, or equivalently `OD(t) = OD_umax * exp(μ_max * (t - t_umax))`

In [3]:
def plot_growth_stats_comparison(stats_dict, title, metric_order=None):
    df = pd.DataFrame(stats_dict).T

    default_metrics = [
        "specific_growth_rate",
        "intrinsic_growth_rate",
        "doubling_time",
        "time_at_umax",
        "exp_phase_start",
        "exp_phase_end",
        "model_rmse",
    ]

    metrics = metric_order or [m for m in default_metrics if m in df.columns]
    numeric_df = df.copy()
    for m in metrics:
        numeric_df[m] = pd.to_numeric(numeric_df[m], errors="coerce")

    n_metrics = len(metrics)
    n_cols = 3
    n_rows = int(np.ceil(n_metrics / n_cols))

    fig = make_subplots(
        rows=n_rows,
        cols=n_cols,
        subplot_titles=[m.replace("_", " ").title() for m in metrics],
        horizontal_spacing=0.08,
        vertical_spacing=0.15,
    )

    method_names = list(numeric_df.index)
    for i, metric in enumerate(metrics):
        row = i // n_cols + 1
        col = i % n_cols + 1
        fig.add_trace(
            go.Bar(
                x=method_names,
                y=numeric_df[metric].tolist(),
                showlegend=False,
                marker=dict(line=dict(color="black", width=1)),
            ),
            row=row,
            col=col,
        )

    fig.update_layout(
        title=title,
        height=max(420, 320 * n_rows),
        width=1200,
        bargap=0.25,
        template="plotly_white",
    )
    return fig

# Mechanistic Models

Mechanistic models are ODE-based parametric models that encode growth dynamics as differential equations. These models are used to model underlying growth processes and can be used to determine the intrinsic growth rate (μ) and carrying capacity (K) of a strain.

# Fit models to the data

Each fit returns a dictionary containing:
- `model_type`: The fitting method used
- `params`: Fitted model parameters (dictionary)

In [20]:
# Generate each fit independently
fit_mech_logistic = gc.parametric.fit_parametric(time, data, method="mech_logistic")
fit_mech_gompertz = gc.parametric.fit_parametric(time, data, method="mech_gompertz")
fit_mech_richards = gc.parametric.fit_parametric(time, data, method="mech_richards")
fit_mech_baranyi = gc.parametric.fit_parametric(time, data, method="mech_baranyi")

# Combine fits into a dictionary for easier iteration
mechanistic_fits = {
    "mech_logistic": fit_mech_logistic,
    "mech_gompertz": fit_mech_gompertz,
    "mech_richards": fit_mech_richards,
    "mech_baranyi": fit_mech_baranyi,
}

# Access and display a single fit result
print("=== Logistic Fit Result ===")
pprint.pprint(fit_mech_logistic)

print("\n=== Accessing specific values ===")
print(f"Model type: {fit_mech_logistic['model_type']}")
print(f"Parameters: {fit_mech_logistic['params']}")

print("\n=== Accessing individual parameters ===")
params = fit_mech_logistic["params"]
print(f"Growth rate (μ): {params['mu']:.6f}")
print(f"Carrying capacity (K): {params['K']:.6f}")
print(f"Initial OD (N0): {params['N0']:.6f}")

=== Logistic Fit Result ===
{'model_type': 'mech_logistic',
 'params': {'K': np.float64(0.4199540943446524),
            'N0': np.float64(1.8464709479959055e-06),
            'fit_t_max': 87.4,
            'fit_t_min': 0.0,
            'mu': np.float64(0.2634572595992891),
            'y0': np.float64(0.05196698646330492)}}

=== Accessing specific values ===
Model type: mech_logistic
Parameters: {'mu': np.float64(0.2634572595992891), 'K': np.float64(0.4199540943446524), 'N0': np.float64(1.8464709479959055e-06), 'y0': np.float64(0.05196698646330492), 'fit_t_min': 0.0, 'fit_t_max': 87.4}

=== Accessing individual parameters ===
Growth rate (μ): 0.263457
Carrying capacity (K): 0.419954
Initial OD (N0): 0.000002


# Extract growth stats from the fit object

In [24]:
# Extract stats from each fit independently
stats_mech_logistic = gc.utils.extract_stats(fit_mech_logistic, time, data)
stats_mech_gompertz = gc.utils.extract_stats(fit_mech_gompertz, time, data)
stats_mech_richards = gc.utils.extract_stats(fit_mech_richards, time, data)
stats_mech_baranyi = gc.utils.extract_stats(fit_mech_baranyi, time, data)

# Combine stats into a dictionary
mechanistic_stats = {
    "mech_logistic": stats_mech_logistic,
    "mech_gompertz": stats_mech_gompertz,
    "mech_richards": stats_mech_richards,
    "mech_baranyi": stats_mech_baranyi,
}

# Display growth statistics for logistic fit
print("=== Logistic Growth Statistics ===")
pprint.pprint(stats_mech_logistic)

# compare stats in a dataframe
mechanistic_df = pd.DataFrame(mechanistic_stats).T[
    [
        "specific_growth_rate",
        "intrinsic_growth_rate",
        "doubling_time",
        "time_at_umax",
        "exp_phase_start",
        "exp_phase_end",
        "model_rmse",
    ]
]
mechanistic_df

=== Logistic Growth Statistics ===
{'doubling_time': 5.207572043167674,
 'exp_phase_end': 56.00576093117492,
 'exp_phase_start': 29.711002069142133,
 'fit_method': 'model_fitting_mech_logistic',
 'fit_t_max': 87.4,
 'fit_t_min': 0.0,
 'intrinsic_growth_rate': 0.2634572595992891,
 'max_od': 0.47192108080795736,
 'model_rmse': 0.005409160816318739,
 'od_at_umax': 0.15769435217401157,
 'specific_growth_rate': 0.13310371413283725,
 'time_at_umax': 42.9118236472946}


,specific_growth_rate,intrinsic_growth_rate,doubling_time,time_at_umax,exp_phase_start,exp_phase_end,model_rmse
mech_logistic,0.133104,0.263457,5.207572,42.911824,29.711002,56.005761,0.005409
mech_gompertz,0.082055,0.049584,8.447369,24.345892,6.773672,71.19749,0.045415
mech_richards,0.125975,0.217648,5.502276,43.437275,28.098186,55.895645,0.004954
mech_baranyi,0.132389,0.263518,5.235674,42.911824,29.665206,56.05596,0.005408


# Visualize the fitted mehcnistic models over the data

In [ ]:
# Create subplots for each mechanistic model

fig_mech = make_subplots(
    rows=2,
    cols=2,
    subplot_titles=("Logistic", "Gompertz", "Richards", "Baranyi"),
    vertical_spacing=0.12,
    horizontal_spacing=0.10,
)

# Define subplot positions
subplot_positions = [
    (1, 1),  # mech_logistic
    (1, 2),  # mech_gompertz
    (2, 1),  # mech_richards
    (2, 2),  # mech_baranyi
]

# Plot each mechanistic model in its own subplot
for (model_name, fit), (row, col) in zip(mechanistic_fits.items(), subplot_positions):
    stats = mechanistic_stats[model_name]

    # Add data points to this subplot
    fig_mech.add_trace(
        go.Scatter(
            x=time,
            y=data,
            mode="markers",
            marker=dict(size=3, opacity=0.3, color="gray"),
            showlegend=False,
        ),
        row=row,
        col=col,
    )

    # Annotate with fit and growth statistics
    fig_mech = gc.plot.annotate_plot(
        fig_mech,
        scale="linear",
        phase_boundaries=(stats["exp_phase_start"], stats["exp_phase_end"]),
        time_umax=stats["time_at_umax"],
        od_umax=stats["od_at_umax"],
        od_max=stats["max_od"],
        umax=stats["specific_growth_rate"],
        umax_point=(stats["time_at_umax"], stats["od_at_umax"]),
        fitted_model=fit,
        row=row,
        col=col,
    )

# Update axes for all subplots
fig_mech.update_xaxes(title_text="Time (hours)", row=2, col=1)
fig_mech.update_xaxes(title_text="Time (hours)", row=2, col=2)
fig_mech.update_yaxes(title_text="OD", row=1, col=1)
fig_mech.update_yaxes(title_text="OD", row=2, col=1)

# Update layout
fig_mech.update_layout(
    title_text="Mechanistic Model Fits with Growth Statistics",
    height=800,
    width=1200,
    showlegend=False,
    template="plotly_white",
)

fig_mech.show()

# Mechanistic Growth Statistics Comparison (Plotly)

This chart compares key growth statistics across mechanistic models.

In [8]:
fig_mech_stats = plot_growth_stats_comparison(
    mechanistic_stats,
    title="Mechanistic models: growth statistics comparison",
)
fig_mech_stats.show()

# Phenomenological Models

Phenomenological approaches focus on fitting observed growth behavior directly, without explicitly modeling underlying mechanisms. These models can be used to calculate descriptive statistics such as the maximum specific growth rate (maximum per capita growth rate, μ_max) but not the intrinsic growth rate.

# Parametric Models

These are phenomenological **parametric** models fit in ln-space.

# Fit models

In [26]:
# Generate each fit independently
fit_phenom_logistic = gc.parametric.fit_parametric(time, data, method="phenom_logistic")
fit_phenom_gompertz = gc.parametric.fit_parametric(time, data, method="phenom_gompertz")
fit_phenom_gompertz_modified = gc.parametric.fit_parametric(
    time, data, method="phenom_gompertz_modified"
)
fit_phenom_richards = gc.parametric.fit_parametric(time, data, method="phenom_richards")

# Combine fits into a dictionary
phenom_param_fits = {
    "phenom_logistic": fit_phenom_logistic,
    "phenom_gompertz": fit_phenom_gompertz,
    "phenom_gompertz_modified": fit_phenom_gompertz_modified,
    "phenom_richards": fit_phenom_richards,
}

# Example: Compare phenomenological logistic fit and stats
print("=== Phenomenological Logistic Fit ===")
pprint.pprint(fit_phenom_logistic)

=== Phenomenological Logistic Fit ===
{'model_type': 'phenom_logistic',
 'params': {'A': np.float64(2.2053922567445503),
            'N0': np.float64(0.052075083822616296),
            'fit_t_max': 87.4,
            'fit_t_min': 0.0,
            'lam': np.float64(34.77980413414468),
            'mu_max': np.float64(0.1357663399984608)}}


# Extract stats from fitted models

In [27]:
# Extract stats from each fit independently
stats_phenom_logistic = gc.utils.extract_stats(fit_phenom_logistic, time, data)
stats_phenom_gompertz = gc.utils.extract_stats(fit_phenom_gompertz, time, data)
stats_phenom_gompertz_modified = gc.utils.extract_stats(
    fit_phenom_gompertz_modified, time, data
)
stats_phenom_richards = gc.utils.extract_stats(fit_phenom_richards, time, data)

# Combine stats into a dictionary
phenom_param_stats = {
    "phenom_logistic": stats_phenom_logistic,
    "phenom_gompertz": stats_phenom_gompertz,
    "phenom_gompertz_modified": stats_phenom_gompertz_modified,
    "phenom_richards": stats_phenom_richards,
}

phenom_param_df = pd.DataFrame(phenom_param_stats).T[
    [
        "specific_growth_rate",
        "intrinsic_growth_rate",
        "doubling_time",
        "time_at_umax",
        "exp_phase_start",
        "exp_phase_end",
        "model_rmse",
    ]
]

print("\n=== Phenomenological Logistic Stats ===")
pprint.pprint(stats_phenom_logistic)

phenom_param_df


=== Phenomenological Logistic Stats ===
{'doubling_time': 5.105442045265444,
 'exp_phase_end': 51.02354925004566,
 'exp_phase_start': 34.77980413414468,
 'fit_method': 'model_fitting_phenom_logistic',
 'fit_t_max': 87.4,
 'fit_t_min': 0.0,
 'intrinsic_growth_rate': None,
 'max_od': 0.4913,
 'model_rmse': 0.005434496057147315,
 'od_at_umax': 0.1570777840090588,
 'specific_growth_rate': 0.1357663399984608,
 'time_at_umax': 42.9118236472946}


,specific_growth_rate,intrinsic_growth_rate,doubling_time,time_at_umax,exp_phase_start,exp_phase_end,model_rmse
phenom_logistic,0.135766,None,5.105442,42.911824,34.779804,51.023549,0.005434
phenom_gompertz,0.160909,None,4.307694,41.160321,36.223882,49.641522,0.007716
phenom_gompertz_modified,0.164217,None,4.220921,41.335471,36.564612,49.620311,0.007574
phenom_richards,0.330063,None,2.100045,43.612425,34.165936,46.683896,0.005156


# Visualize fits and calculated growth descriptors

In [ ]:
# Create subplots for each phenomenological parametric model
fig_phenom_param = make_subplots(
    rows=2,
    cols=2,
    subplot_titles=("Logistic", "Gompertz", "Gompertz (modified)", "Richards"),
    vertical_spacing=0.12,
    horizontal_spacing=0.10,
)

# Define subplot positions
subplot_positions = [
    (1, 1),  # phenom_logistic
    (1, 2),  # phenom_gompertz
    (2, 1),  # phenom_gompertz_modified
    (2, 2),  # phenom_richards
]

# Plot each phenomenological parametric model in its own subplot
for (model_name, fit), (row, col) in zip(phenom_param_fits.items(), subplot_positions):
    stats = phenom_param_stats[model_name]

    # Add data points to this subplot
    fig_phenom_param.add_trace(
        go.Scatter(
            x=time,
            y=data,
            mode="markers",
            marker=dict(size=3, opacity=0.3, color="gray"),
            showlegend=False,
        ),
        row=row,
        col=col,
    )

    # Annotate with fit and growth statistics
    fig_phenom_param = gc.plot.annotate_plot(
        fig_phenom_param,
        scale="linear",
        phase_boundaries=(stats["exp_phase_start"], stats["exp_phase_end"]),
        time_umax=stats["time_at_umax"],
        od_umax=stats["od_at_umax"],
        od_max=stats["max_od"],
        umax=stats["specific_growth_rate"],
        umax_point=(stats["time_at_umax"], stats["od_at_umax"]),
        fitted_model=fit,
        row=row,
        col=col,
    )

# Update axes for all subplots
fig_phenom_param.update_xaxes(title_text="Time (hours)", row=2, col=1)
fig_phenom_param.update_xaxes(title_text="Time (hours)", row=2, col=2)
fig_phenom_param.update_yaxes(title_text="OD", row=1, col=1)
fig_phenom_param.update_yaxes(title_text="OD", row=2, col=1)

# Update layout
fig_phenom_param.update_layout(
    title_text="Phenomenological Parametric Model Fits with Growth Statistics",
    height=800,
    width=1200,
    showlegend=False,
    template="plotly_white",
)

fig_phenom_param.show()

# Non-Parametric Methods

These are phenomenological **non-parametric** fits that estimate growth features directly from local trends and smoothing. Spline and sliding window options are available. These methods only fit small sections of the data (sliding_window: linear model of Ln(N) across a user defined window of points; spline: spline curve fitted to the region around the exponential phase).

In [28]:
# Generate each fit independently
fit_spline = gc.non_parametric.fit_non_parametric(
    time,
    data,
    method="spline",
    spline_s=0.001,
)

fit_sliding_window = gc.non_parametric.fit_non_parametric(
    time,
    data,
    method="sliding_window",
    window_points=7,
)

# Combine fits into a dictionary
phenom_nonparam_fits = {
    "spline": fit_spline,
    "sliding_window": fit_sliding_window,
}

# Display non-parametric fit and stats results
pprint.pprint(phenom_nonparam_fits)

{'sliding_window': {'model_type': 'sliding_window',
                    'params': {'fit_t_max': 45.4,
                               'fit_t_min': 44.2,
                               'intercept': -8.295980209369276,
                               'slope': 0.14867963611003765,
                               'time_at_umax': 44.8,
                               'window_points': 7}},
 'spline': {'model_type': 'spline',
            'params': {'fit_t_max': 58.0,
                       'fit_t_min': 19.2,
                       'mu_max': 0.144848579565168,
                       'spline_s': 0.001,
                       'tck_c': [-2.9507908047913984,
                                 -2.9251132997765428,
                                 -2.8881378965136273,
                                 -2.771802491225891,
                                 -2.56913323327732,
                                 -2.3553128078543124,
                                 -2.192414965043854,
                             

# Extract stats from method object

In [29]:
# Extract stats from each fit independently
stats_spline = gc.utils.extract_stats(
    fit_spline,
    time,
    data,
    phase_boundary_method="tangent",
)

stats_sliding_window = gc.utils.extract_stats(
    fit_sliding_window,
    time,
    data,
    phase_boundary_method="tangent",
)

# Combine stats into a dictionary
phenom_nonparam_stats = {
    "spline": stats_spline,
    "sliding_window": stats_sliding_window,
}

phenom_nonparam_df = pd.DataFrame(phenom_nonparam_stats).T[
    [
        "specific_growth_rate",
        "intrinsic_growth_rate",
        "doubling_time",
        "time_at_umax",
        "exp_phase_start",
        "exp_phase_end",
        "model_rmse",
    ]
]
phenom_nonparam_df

,specific_growth_rate,intrinsic_growth_rate,doubling_time,time_at_umax,exp_phase_start,exp_phase_end,model_rmse
spline,0.144849,None,4.785323,44.936683,35.233618,51.162511,0.002795
sliding_window,0.14868,None,4.662018,44.8,35.499161,51.017611,0.001146


In [ ]:
# create the base plot with data
fig_spline = gc.plot.create_base_plot(time, data, scale="linear")

# annotate the plot with growth statistics and fitted model
fig_spline = gc.plot.annotate_plot(
    fig_spline,
    scale="linear",
    phase_boundaries=(stats_spline["exp_phase_start"], stats_spline["exp_phase_end"]),
    time_umax=stats_spline["time_at_umax"],
    od_umax=stats_spline["od_at_umax"],
    od_max=stats_spline["max_od"],
    umax=stats_spline["specific_growth_rate"],
    umax_point=(stats_spline["time_at_umax"], stats_spline["od_at_umax"]),
    fitted_model=phenom_nonparam_fits["spline"],
)

# add title and show
fig_spline.update_layout(title="Phenomenological non-parametric example (spline)")
fig_spline.show()

### Quick Comparison Across Phenomenological Approaches

This table combines phenomenological parametric and non-parametric methods.

In [15]:
phenom_all_stats = {
    **phenom_param_stats,
    **phenom_nonparam_stats,
}

phenom_all_df = pd.DataFrame(phenom_all_stats).T[
    [
        "specific_growth_rate",
        "doubling_time",
        "time_at_umax",
        "exp_phase_start",
        "exp_phase_end",
        "model_rmse",
        "fit_method",
    ]
]
phenom_all_df

,specific_growth_rate,doubling_time,time_at_umax,exp_phase_start,exp_phase_end,model_rmse,fit_method
phenom_logistic,0.135766,5.105442,42.911824,34.779804,51.023549,0.005434,model_fitting_phenom_logistic
phenom_gompertz,0.160909,4.307694,41.160321,36.223882,49.641522,0.007716,model_fitting_phenom_gompertz
phenom_gompertz_modified,0.164217,4.220921,41.335471,36.564612,49.620311,0.007574,model_fitting_phenom_gompertz_modified
phenom_richards,0.330063,2.100045,43.612425,34.165936,46.683896,0.005156,model_fitting_phenom_richards
spline,0.144849,4.785323,44.936683,35.233618,51.162511,0.002795,model_fitting_spline
sliding_window,0.14868,4.662018,44.8,35.499161,51.017611,0.001146,model_fitting_sliding_window


### Phenomenological Growth Statistics Comparison (Plotly)

This chart compares key growth statistics across phenomenological parametric and non-parametric approaches.

In [16]:
fig_phenom_stats = plot_growth_stats_comparison(
    phenom_all_stats,
    title="Phenomenological models: growth statistics comparison",
)
fig_phenom_stats.show()